
### Examining racial discrimination in the US job market

#### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

#### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes.

#### Exercise
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Discuss statistical significance.

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [27]:
import pandas as pd
import numpy as np
import numpy.random as npr
from scipy import stats
import matplotlib.pyplot as plt

In [9]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
data.head()


,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


In [8]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [20]:
# number of callbacks for black-sounding names
blackcall = sum(data[data.race=='b'].call)

In [21]:
# number of callbacks for white-sounding names
whitecall = sum(data[data.race=='w'].call)

In [25]:
blackpop = len(data[data.race=='b'])
whitepop = len(data[data.race=='b'])
ratioblack = blackcall/blackpop
ratiowhite = whitecall/whitepop
print(ratioblack*100,ratiowhite*100)

6.4476386037 9.65092402464


In [48]:
print(len(data))

def bootstrap(data, num_samples, statistic, alpha):
    """Returns bootstrap estimate of 100.0*(1-alpha) CI for statistic."""
    n = len(data)
    idx = npr.randint(0, n, (num_samples, n))
    samples = data[idx]
    stat = np.sort(statistic(samples, 1))
    return (stat[int((alpha/2.0)*num_samples)],
            stat[int((1-alpha/2.0)*num_samples)])


4870


In [70]:
popcall = np.asarray(data.call)
low, high = bootstrap(popcall, 10000, np.mean, 1.0) #gives the mean
print(low*100)
low, high = bootstrap(popcall, 10000, np.mean, 0.05) #gives 95% confidence level
print(low*100,high*100)


8.04928168654
7.28952735662 8.80903527141


   1. What test is appropriate for this problem? Does CLT apply?
   
Since we have no information about the population mean, we need to employ bootstrapping.  

   2. What are the null and alternate hypotheses?

The null hypothesis is that the the callback rate (defined as number of callbacks divided by population for a particular rate) is the same regardless of race.

The alternative hypothesis is that the callback rate is affected by the race.  

   3. Compute margin of error, confidence interval, and p-value.

The 95% confidence interval is [7.290, 8.809] and the margin of error is 0.76 which corresponds to a std error of 0.76/1.96 = .388.  With a callback rate of 6.448 for blacks, this is a z-score of -4.126 which yields a p-value that is REALLY LOW, basically zero.  The same can be said of the callback rate for whites.  Here the z-score is (9.651-8.049)/.388 = 4.129 yielding a p-value of near zero.

   4. Discuss statistical significance.


Given that the p-value is well below any acceptable critical value, this leads us to believe we can reject the null hypothesis that callback rates are not affected by race.  